In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/Colab Notebooks/OursRepository/public-opinion-monitor

!pip install thulac
!pip install jieba

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/OursRepository/public-opinion-monitor
     |████████████████████████████████| 52.9MB 61kB/s 
  Created wheel for thulac: filename=thulac-0.2.1-cp36-none-any.whl size=53141671 sha256=b8e133500d760953064fc20ad0eaf069243c7312c90850b1df3d2ac4d720d216
  Stored in directory: /root/.cache/pip/wheels/db/36/4a/1ac1e9b9ce727a9dfc7fa20092992707d7da162df871c8488f
Successfully built thulac


In [2]:
from HANModel import *
import torch
import torch.nn as nn
from torch.optim.optimizer import Optimizer
import numpy
import time, math
import torch.utils.data as data
import os
import pandas as pd
import json
import thulac
thulac = thulac.thulac()
import jieba

Model loaded succeed


In [3]:
# import torch.functional as F
# 
embedding = nn.Embedding(10, 3)
input = torch.LongTensor([[0, 1,2,4,5],[1, 4,3,2,9]])
print(embedding(input))
embedding2 = nn.Embedding(10, 3, padding_idx=2)
# input = torch.LongTensor([[1,2,0,5, 6,7,8,9]])
print(embedding2(input))
weight = torch.FloatTensor([[1, 2.3, 3], [4, 5.1, 6.3]])
embedding3 = nn.Embedding.from_pretrained(weight)
input = torch.LongTensor([1])
print(embedding3(input))
input = torch.LongTensor([0])
print(embedding3(input))



tensor([[[-0.0518,  0.2434, -1.3075],
         [-0.4011,  0.2220, -2.3050],
         [ 1.0652, -1.1514, -0.5745],
         [-0.6423, -0.5730, -0.8868],
         [ 0.0924,  0.8747, -0.3405]],

        [[-0.4011,  0.2220, -2.3050],
         [-0.6423, -0.5730, -0.8868],
         [-0.8263, -0.3307,  0.9537],
         [ 1.0652, -1.1514, -0.5745],
         [-0.9568, -0.5339, -0.3380]]], grad_fn=<EmbeddingBackward>)
tensor([[[ 1.2811,  0.2928, -0.6268],
         [-0.9384, -0.7297, -0.8988],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.4952, -0.2101,  1.0487],
         [ 0.1760, -0.1727, -0.6746]],

        [[-0.9384, -0.7297, -0.8988],
         [ 0.4952, -0.2101,  1.0487],
         [-0.4240,  0.8372, -1.6866],
         [ 0.0000,  0.0000,  0.0000],
         [-1.3208,  0.5617,  0.5079]]], grad_fn=<EmbeddingBackward>)
tensor([[4.0000, 5.1000, 6.3000]])
tensor([[1.0000, 2.3000, 3.0000]])


## 读取词向量
建立词语列表

In [4]:
from gensim.models import KeyedVectors, Word2Vec

# file = '../../PretrainedData/Tencent_AILab_ChineseEmbedding/Tencent_AILab_ChineseEmbedding.txt'
# file = '../../DataSets/Word2Vect/xingrong_50_thulac/word2vect_50_w5.model'
file = '../../DataSets/Word2Vect/xiejunjie_300_jieba/wiki_han_word2vec_300维度.model'
word2vec = Word2Vec.load(file)
# word2vec = KeyedVectors.load_word2vec_format(file, binary=False)
word2vec.init_sims(replace=True)  # 神奇，很省内存，可以运算most_similar
word2vec.vector_size

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


300

In [5]:
print(word2vec)
# print(word2vec.wv.vocab)
# print(len(word2vec.index2word))
print(len(word2vec.wv.index2word))
print(word2vec.wv.index2word[0])
print(word2vec.wv.index2word[1])
print(word2vec.wv.index2word[2])
print(word2vec.wv.index2word[1522])
print(word2vec.wv.index2entity[1522])
print(word2vec.similar_by_word('中国'))
print(word2vec.similar_by_word('天才'))
print(word2vec.wv)
print('word2vec.wv.vocab ---- >', word2vec.wv.vocab)
print(word2vec.wv.index2word)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be remo

['', '的', '在', '年', '是', '为', '和', '月', '了', '于', '与', '有', '被', '他', '中', '等', '日', '中国', '也', '并', '后', '由', '而', '及', '以', '上', '一个', '人', '至', '但', '将', '分布', '对', '其', '到', '1', '学名', '时', '属', '以及', '都', '或', '米', '会', '地', '2', '3', '日本', '地区', '可以', '不', '4', '该', '台湾', '植物', '之', '大陆', '主要', '5', '开始', '从', '10', '成为', '个', '美国', '6', '位于', '8', '则', '使用', '进行', '号', '目前', '包括', '7', '这', '就', '第', '12', '所', '下', '生长', '9', '没有', '曾', '又', '他们', '由于', '香港', '其中', '大', '11', '新', '可', '一种', '多', '2009', '亦', '她', '站', '前', '国家', '认为', '因此', '其他', '向', '已', '当时', '作为', '之后', '来', '因', '它', '获得', '还', '海拔', '物种', '因为', '更', '内', '自己', '人工', '公司', '最', '栽培', '同时', '引种', '通过', '尚未', '之一', '2010', '车站', '约', '模式', '/', '世界', '政府', '一些', '20', '生活', '成立', '可能', '—', '们', '大学', '型', '一', '使', '这个', '不同', '电影', '动物', '发展', '国际', '最后', '让', '一般', '2008', '部分', '郡', '今', '区', '名', '后来', '称为', '出现', '时间', '把', '代表', '发现', '工作', '但是', '已经', '这些', '生于', '担任', '要', '说', '即', '此', '比赛', '指',

In [6]:
wordEmbedding = [word2vec.wv[word]  for word in word2vec.wv.index2word]
word2index = { word:i for i, word in enumerate(word2vec.wv.index2word)}
print(wordEmbedding[:10])
print(word2index['中国'])
print(word2index['天才'])


[array([ 6.30656257e-02, -1.12314887e-01,  4.85262945e-02, -8.55197757e-02,
        4.55908328e-02,  6.46558106e-02, -4.27534170e-02,  5.36949895e-02,
       -2.26631403e-01,  6.25260845e-02,  1.83971524e-02, -3.61814126e-02,
       -7.80684203e-02, -9.78453308e-02,  3.07323225e-02,  4.43015546e-02,
       -1.03984829e-02, -3.16940509e-02, -4.82366160e-02, -2.55376995e-02,
       -1.94135922e-04, -2.42569018e-02, -5.79886399e-02,  1.26771117e-02,
        7.19933212e-03, -5.69830649e-02, -1.26355989e-02,  2.16077268e-02,
       -8.15668255e-02,  9.75341350e-03,  4.03154232e-02, -2.12772768e-02,
        4.27430905e-02, -2.53562219e-02,  9.61699188e-02,  4.80885766e-02,
       -5.42805195e-02, -3.23644653e-02, -1.98951829e-02,  3.14243254e-03,
       -4.23267344e-03,  8.15014020e-02,  5.07650971e-02, -6.43669665e-02,
        5.04725389e-02,  2.95574106e-02, -1.39722880e-02, -9.62358862e-02,
       -4.40400690e-02, -2.26076338e-02, -4.84022610e-02,  6.48457836e-03,
       -5.65880351e-02, 

## 读取训练数据

In [7]:
def compute_ngrams(word, num_min = 1, num_max = 3):
    ngrams =[]
    for ngram_length in range(num_min, min(len(word), num_max) + 1):
        for i in range(len(word) - ngram_length + 1):
            # print(i, i + ngram_length)
            ngrams.append(word[i : i + ngram_length])
    # print(ngrams)
    return list(set(ngrams))

print(compute_ngrams('you'))
print(compute_ngrams('I Think'))
print(compute_ngrams('中华人民共和国万岁'))

['yo', 'u', 'o', 'y', 'you', 'ou']
[' T', 'nk', 'I', 'Th', 'n', 'I ', 'T', 'I T', ' Th', 'hin', 'ink', 'k', 'Thi', 'in', 'hi', 'i', ' ', 'h']
['华人', '中华人', '华', '民共', '和国', '人民', '人', '国万', '和', '华人民', '万岁', '岁', '中华', '民', '共和国', '共和', '国万岁', '人民共', '中', '国', '万', '民共和', '和国万', '共']


In [0]:
# 从词向量文本文件 word2vec 中获取词向量，如果获取到直接返回，若没有获取到，那么把这个词拆开
# 成为 ngrams 的新词组，并在 word2vec 中找新词组中的词向量并相加取平均，最后得到平均词向量输出
def wordVec(word, word2vec, min_n = 1, max_n = 3):
    # 确认词向量维度
    word_size = word2vec.wv.syn0[0].shape[0]

    # 如果在词典之中，直接返回词向量
    if word in word2vec.wv.vocab.keys():
        return word2vec[word]
    else:
        # 计算word的ngrams词组
        ngrams = compute_ngrams(word, min_n, max_n)
        # 不在词典的情况下
        word_vec = numpy.zeros(word_size, dtype=numpy.float32)
        ngrams_found = 0
        ngrams_single = [ng for ng in ngrams if len(ng) == 1]
        ngrams_more = [ng for ng in ngrams if len(ng) > 1]
        # 先只接受2个单词长度以上的词向量
        for ngram in ngrams_more:
            if ngram in word2vec.wv.vocab.keys():
                word_vec += word2vec[ngram]
                ngrams_found += 1
                #print(ngram)
        # 如果，没有匹配到，那么最后是考虑单个词向量
        if ngrams_found == 0:
            for ngram in ngrams_single:
                word_vec += word2vec[ngram]
                ngrams_found += 1
        if word_vec.any():
            return word_vec / max(1, ngrams_found)
        else:
            # 不抛出异常，而是打印提示，并返回0向量。
            print(KeyError('all ngrams for word %s absent from model' % word))
            return word_vec

In [0]:
class DotDict(dict):
    def __init__(self, *args, **kwargs):
        dict.__init__(self, *args, **kwargs)
        self.__dict__ = self
        

In [10]:
import pandas as pd

dic = {'a':[1, 2, 3, 4], 'b':[5, 6, 7, 8],
'c':[9, 10, 11, 12], 'd':[13, 14, 15, 16]}
df1=pd.DataFrame(dic)
print(df1)
df2=df1.sample(frac=0.75)
print(df2)
# rowlist=[]
# for indexs in df2.index:
#     rowlist.append(indexs)
df3=df1.drop(df2.index.to_list(),axis=0)
print(df3)

   a  b   c   d
0  1  5   9  13
1  2  6  10  14
2  3  7  11  15
3  4  8  12  16
   a  b   c   d
1  2  6  10  14
0  1  5   9  13
2  3  7  11  15
   a  b   c   d
3  4  8  12  16


In [0]:
def isNan(a):
    return a != a

class RatingData(data.Dataset):
    def __init__(self, path, 
                 word2index, 
                 max_row = -1, 
                 trainTestRate = 0.85, 
                 isTrain = True, 
                 wordCuter = thulac,
                 clean_file_name = 'ratings_clean_4_HAN.csv',
                 ):
        self.token_list = []
        self.label_list = []
        # self.token_positions = torch.tensor([i for i in range(100)])

        print(' balance_data.csv 所在path:',path) # 地址不应该包含 ratings.csv

        ratings_clean_filename = os.path.join(path, clean_file_name)
        ratings_filename = os.path.join(path, 'balance_data.csv')
        if os.path.isfile(ratings_clean_filename):
            clean_pd = pd.read_csv(ratings_clean_filename)
        else:
            print('没有找到缓存的文件%s, 读取源文件%s'%(ratings_clean_filename, ratings_filename))
            ratings_pd = pd.read_csv(ratings_filename)
            print('开始生成缓存文件%s'%(ratings_clean_filename))
            clean_pd = pd.DataFrame({
                'labels':[],
                'data':[],
            })
            nonRatingCount = 0
            for i, row in ratings_pd.iterrows():
                if max_row != -1 and i > max_row:
                    break
                if not isinstance(row['data'], str) or row['data'] == '':
                    # print(i + 1, row['comment'])
                    nonRatingCount += 1
                    continue
                r0 = row['labels']
                if r0 == -1:
                    r0 = 2
                    
                if i % 10000 == 9999:
                    print(i + 1, r0)

                words = list(wordCuter.cut(row['data']))
                # 0 在词向量集中是‘，’，换个词向量集可能表示其他
                token = [ word2index[words[i]] if i < len(words) and words[i] in word2index else 0 
                          for i in range(100)] 
                
                newRow = DotDict()

                newRow.labels = [r0]
                newRow.data = [json.dumps(token)]

                clean_pd = clean_pd.append(pd.DataFrame(newRow), ignore_index=True)
            print('空的评论数量： %d'%(nonRatingCount))
            clean_pd.to_csv(ratings_clean_filename)

        # 读取
        if isTrain:
            temp_pd = clean_pd.sample(frac=trainTestRate)
        else:
            temp_pd = clean_pd.sample(frac=trainTestRate)
            temp_pd = clean_pd.drop(temp_pd.index.tolist(), axis=0)

        for i, row in temp_pd.iterrows():
            if max_row != -1 and i > max_row:
                break

            self.label_list.append(torch.tensor(row['labels']).long())
            self.token_list.append(torch.from_numpy(numpy.array( json.loads(row['data']) ) ).long())

    def __getitem__(self, index):
        return self.token_list[index], self.label_list[index]#, self.token_positions

    def __len__(self):
        return len(self.label_list)
    

In [12]:
ratingData = RatingData('../../DataSets/yf_dianping',
                             word2index = word2index,
                             # max_row= 200000,
                             isTrain=True,
                             trainTestRate = 0.8,
                             wordCuter= jieba,
                             clean_file_name='balance_data_4_HAN.csv',
                             )
trainLoader = torch.utils.data.DataLoader(dataset=ratingData,
                                          batch_size=128,
                                          shuffle = True,
                                          num_workers = 8,
                                          )
ratingData2 = RatingData('../../DataSets/yf_dianping',
                             word2index = word2index,
                             # max_row= 200000,
                             isTrain=False,
                             trainTestRate = 0.8,
                             wordCuter= jieba,
                             clean_file_name='balance_data_4_HAN.csv',
                             )
testLoader = torch.utils.data.DataLoader(dataset=ratingData2,
                                          batch_size=128,
                                          shuffle = True,
                                          num_workers = 8,
                                          )
print(len(ratingData.label_list))
print(len(ratingData2.label_list))


 balance_data.csv 所在path: ../../DataSets/yf_dianping
 balance_data.csv 所在path: ../../DataSets/yf_dianping
530311
132578


In [13]:
#创建模型
wordEmbedding = torch.FloatTensor(wordEmbedding)
num_embeddings = len(word2vec.wv.index2word)
model = HAN(num_embeddings, 
            num_classes = 3,
            embedding_dim=word2vec.wv.vector_size, 
            num_words = 100,
            hidden_size_gru = 300,
            hidden_size_att = 600,
            )
print(model)

modelParams = model.parameters()
for param in modelParams:
    if len(param.data.shape) > 1:
        print('---', param.data.shape, param.data)
        torch.nn.init.kaiming_normal(param.data)
        print('--->', param.data)
        
model.embed.from_pretrained(wordEmbedding)


HAN(
  (embed): Embedding(109683, 300, padding_idx=0)
  (GRU1): GRU(300, 300, batch_first=True, bidirectional=True)
  (self_attention1): SelfAttention(
    (W): Linear(in_features=600, out_features=600, bias=True)
    (U): Linear(in_features=600, out_features=1, bias=True)
  )
  (GRU2): GRU(600, 300, batch_first=True, bidirectional=True)
  (self_attention2): SelfAttention(
    (W): Linear(in_features=600, out_features=600, bias=True)
    (U): Linear(in_features=600, out_features=1, bias=True)
  )
  (fc): Linear(in_features=600, out_features=3, bias=True)
)
--- torch.Size([109683, 300]) tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 1.8740,  1.2330, -1.4282,  ...,  0.7074, -1.2020, -0.6084],
        [ 0.8966, -0.7228, -0.0610,  ..., -0.5450, -0.3052,  0.3359],
        ...,
        [-0.8745, -1.3674, -0.8517,  ...,  0.4811, -1.4489, -0.2738],
        [ 0.2816, -1.2541, -0.3038,  ..., -2.0078, -0.4436, -1.3187],
        [-0.1475,  0.8429, -0.4614,  ..., -0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  app.launch_new_instance()


---> tensor([[-0.0228,  0.0134, -0.0093,  ..., -0.0362, -0.0573, -0.0525],
        [ 0.0150,  0.0541,  0.0733,  ...,  0.0245, -0.0847, -0.0180],
        [-0.0470,  0.0904,  0.1457,  ...,  0.0170, -0.0912,  0.0137],
        ...,
        [ 0.0973,  0.1756, -0.0016,  ...,  0.0645,  0.0482, -0.0567],
        [ 0.0308, -0.0362,  0.0157,  ...,  0.1631, -0.0908, -0.0541],
        [ 0.1153,  0.0154,  0.0495,  ..., -0.0247,  0.0956,  0.1393]])
--- torch.Size([900, 300]) tensor([[-0.0487, -0.0406, -0.0321,  ...,  0.0289,  0.0264, -0.0510],
        [-0.0472,  0.0331, -0.0110,  ...,  0.0331,  0.0345, -0.0386],
        [-0.0433, -0.0147, -0.0220,  ..., -0.0043,  0.0124, -0.0251],
        ...,
        [ 0.0480,  0.0553, -0.0543,  ...,  0.0154, -0.0132,  0.0403],
        [ 0.0196,  0.0395, -0.0407,  ...,  0.0558,  0.0105, -0.0128],
        [ 0.0026, -0.0361,  0.0385,  ...,  0.0439,  0.0349, -0.0205]])
---> tensor([[-0.0741, -0.0184, -0.0351,  ...,  0.0551,  0.0444,  0.0710],
        [-0.0702, -0.0150

Embedding(109683, 300)

## 开始训练过程

In [0]:
def trainOneEpoch(epoch, model:HAN, trainLoader, optimizer:Optimizer, lossFunc):
    if torch.cuda.is_available():
        model = model.cuda()
        lossFunc = lossFunc.cuda()

    model.train()

    startTime = time.time()
    for i, (x, y) in enumerate(trainLoader):
        if torch.cuda.is_available():
            x = x.cuda()
            y = y.cuda()

        outputs = model(x)
        loss = lossFunc(outputs, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 100 == 99:
            print('Epoch %d, %d/%d, loss:%f ' % (epoch, i, len(trainLoader), loss))
        # if i > 2000:
        #     break 
    print('Epoch %d cost time: %.3fs' % (epoch, time.time() - startTime))


def testModel(epoch, model:HAN, testLoader):
    if torch.cuda.is_available():
        model = model.cuda()

    model.eval()

    total = 0
    correct = 0

    startTime = time.time()
    for i, (x, y) in enumerate(testLoader):
        if torch.cuda.is_available():
            x = x.cuda()
            y = y.cuda()

        outputs = model(x)
        _, predicted = torch.max(outputs.data, 1)

        total += len(y)
        correct += predicted.data.eq(y.data).cpu().sum().numpy()
        

        if i % 50 == 49:
            print('Epoch Test %d, %d/%d, \npredicted:%s, \ntarget:%s' % (epoch, i, len(testLoader), 
                                                                     str(predicted),
                                                                     str(y)))
        # if i > 2000:
        #     break 
    print('Epoch Test %d cost time: %.3fs' % (epoch, time.time() - startTime))
    print('准确率： %.3f' % (correct / total))
    return correct / total


def train(nepoch, model, modelSavePath, isLoad, lr=0.0002):
    last_acc = 0
    if isLoad: 
      model.load_state_dict(torch.load(modelSavePath))
      last_acc = testModel(epoch, model, testLoader)
    # optimizer=torch.optim.SGD(model.parameters(), momentum=0.9, lr=0.001)
    optimizer=torch.optim.Adam(model.parameters(), lr=0.0002)
    lossFunc =torch.nn.CrossEntropyLoss()
    for epoch in range(nepoch):
        trainOneEpoch(epoch, model, trainLoader, optimizer, lossFunc)
        acc = testModel(epoch, model, testLoader)
        if last_acc < acc:
          torch.save(model.state_dict(), modelSavePath)
        last_acc = acc
    



In [21]:

train(30, model, 'EmotionAnalyzeModelData_300_600.model', True, lr=0.0002)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


流式输出内容被截断，只能显示最后 5000 行内容。
        2, 1, 2, 0, 1, 1, 1, 2, 1, 1, 2, 0, 2, 1, 0, 2, 0, 0, 2, 2, 2, 0, 0, 2,
        2, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 2, 1, 1, 2, 1, 1, 1,
        0, 0, 2, 0, 2, 1, 0, 1], device='cuda:0')
Epoch Test 13, 549/1036, 
predicted:tensor([0, 2, 1, 0, 1, 2, 1, 1, 0, 2, 0, 0, 2, 1, 0, 2, 1, 0, 0, 1, 2, 0, 0, 2,
        1, 0, 2, 1, 1, 2, 2, 1, 0, 2, 0, 0, 2, 0, 2, 1, 1, 0, 2, 1, 2, 0, 1, 0,
        0, 0, 0, 2, 0, 0, 2, 2, 0, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1,
        2, 1, 2, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 2, 2, 0, 2, 0, 0, 1, 1, 0, 2,
        2, 2, 0, 1, 2, 1, 2, 1, 0, 1, 1, 0, 1, 2, 0, 2, 1, 1, 0, 0, 0, 1, 0, 0,
        0, 1, 2, 1, 2, 2, 1, 0], device='cuda:0'), 
target:tensor([0, 2, 1, 0, 1, 2, 1, 1, 0, 1, 1, 0, 2, 1, 0, 0, 1, 0, 1, 1, 2, 0, 0, 2,
        1, 0, 2, 1, 1, 2, 2, 0, 0, 2, 1, 0, 2, 0, 2, 1, 1, 0, 1, 1, 2, 0, 1, 0,
        1, 0, 0, 1, 2, 0, 0, 2, 0, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 2, 1, 1, 2,
        2, 1, 2, 0, 0, 1, 1,

In [17]:

def eval(model, modelSavePath, isLoad = True):
    if isLoad: model.load_state_dict(torch.load(modelSavePath))
    testModel(0, model, testLoader)

eval(model, 'EmotionAnalyzeModelData_300_600.model', isLoad = True)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch Test 0, 49/1036, 
predicted:tensor([1, 0, 0, 1, 2, 0, 2, 0, 0, 2, 2, 0, 2, 1, 1, 1, 1, 0, 1, 2, 2, 2, 2, 0,
        1, 1, 0, 0, 2, 0, 1, 2, 2, 2, 1, 1, 0, 1, 2, 2, 0, 0, 1, 1, 1, 2, 2, 0,
        0, 0, 1, 2, 2, 1, 1, 1, 2, 2, 1, 0, 1, 0, 1, 2, 2, 1, 0, 0, 0, 2, 1, 1,
        2, 2, 1, 1, 2, 1, 2, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1,
        1, 0, 2, 2, 1, 1, 0, 2, 0, 2, 0, 2, 0, 0, 1, 2, 1, 0, 1, 2, 2, 0, 2, 1,
        0, 1, 0, 0, 1, 2, 2, 2], device='cuda:0'), 
target:tensor([1, 1, 0, 2, 2, 0, 2, 1, 0, 2, 2, 0, 2, 1, 1, 1, 1, 0, 1, 2, 2, 2, 2, 0,
        1, 1, 0, 1, 0, 0, 2, 2, 2, 2, 0, 1, 1, 1, 2, 2, 0, 0, 1, 1, 1, 2, 2, 1,
        0, 0, 1, 2, 2, 2, 1, 1, 2, 2, 1, 0, 2, 0, 1, 2, 2, 0, 1, 0, 0, 2, 0, 0,
        2, 2, 1, 1, 2, 1, 2, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 2, 2, 2, 1, 0, 1, 1,
        1, 0, 2, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 1, 0, 2, 2, 2, 0, 2, 1,
        0, 2, 0, 0, 1, 2, 2, 2], device='cuda:0')
Epoch Test 0, 99/1036, 
predicted:tensor([2, 2, 0, 1, 1, 

In [0]:
lossFunc =torch.nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
_, predicted = torch.max(input.data, 1)
output = lossFunc(input, target)
print('input',input, '\n target',target, '\n output', output)
output = lossFunc(input, predicted)
print('predicted',predicted, '\n target',target, '\n output', output)

print(predicted.data.eq(target).cpu().sum())
print(target.data.eq(predicted).cpu().sum())

In [0]:
# 何凯明初始化

w = torch.Tensor(3, 5, 2)
print(w)
print(nn.init.kaiming_uniform(w))
print(w)
w = torch.Tensor(3, 5, 2)
print(w)
print(torch.nn.init.kaiming_normal(w))
print(w)